In [1]:
import torch
import numpy as np
# from mlp-mixer import MLPMixer

audio_output = torch.ones([3, 1024]) # Batch, Seq
text_output = torch.ones([3, 768]) # Batch, Seq

# a.unsqueeze_(2)

# Performs a batch matrix-matrix product of matrices stored in input and mat2.
matmul_output = torch.bmm(audio_output.unsqueeze_(2), text_output.unsqueeze_(1))
print(matmul_output.unsqueeze_(1).shape) # Batch, Color, Width, Hight


torch.Size([3, 1, 1024, 768])


In [5]:
from erc.model import MLPMixer


model = MLPMixer(
    image_size = (1024, 768),
    channels = 1,
    patch_size = 16,
    dim = 512,
    depth = 12,
    num_classes = 7
)


In [6]:
pred = model(matmul_output) 
print(pred.shape)

torch.Size([3, 7])


In [14]:
os.listdir("/home/hoesungryu/workspace/AI-Hub_emotion_dialog")


['annotation', 'wav']

In [16]:
from glob import glob
import os 
PRETRAINED_DATA_PATH = '/home/hoesungryu/workspace/AI-Hub_emotion_dialog'
txt_folder =sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'annotation')+'/*.csv'))
wav_folder = sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'wav')+'/*.wav'))

idx = 0 
print(txt_folder[0], wav_folder[0])



/home/hoesungryu/workspace/AI-Hub_emotion_dialog/annotation/0038_G1A4E2S4C0_OMH_000001.csv /home/hoesungryu/workspace/AI-Hub_emotion_dialog/wav/0038_G1A4E2S4C0_OMH_000001.wav


In [18]:
import pandas as pd
df = pd.read_csv(txt_folder[0])
df

,txt,wav,emotion
0,"왕자는 다시 공주 앞에 나타났지만, 공주는 외면했습니다.",0038_G1A4E2S4C0_OMH_000001.wav,sad


In [25]:
txt, _, emotion, df.iloc[0].values

'왕자는 다시 공주 앞에 나타났지만, 공주는 외면했습니다.'

In [ ]:
import os 
from glob import glob

def get_hub_txt(self, txt: str, encoding: str = None)-> Tuple[torch.Tensor, torch.Tensor]:
    if self.tokenizer:
        result: dict = self.tokenizer(text=txt,
                                    padding="max_length",
                                    truncation="only_first",
                                    max_length=self.max_length_txt,
                                    return_attention_mask=True,
                                    return_tensors="pt")
        input_ids = result["input_ids"].squeeze()
        mask = result["attention_mask"].squeeze()
        return input_ids, mask
    else:
        return txt, None
    
class AIHubDialog():
    # PRETRAINED_DATA_PATH = '/home/hoesungryu/workspace/AI-Hub_emotion_dialog'
    def __init__(self, PRETRAINED_DATA_PATH):
        self.txt_folder = sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'annotation')+'/*.csv'))
        self.wav_folder = sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'wav')+'/*.wav'))

    def __len__(self):
        assert len(glob(self.wav_folder)) == len(glob(self.txt_folder))
        return len(glob(self.wav_folder)) 
    
    def __getitem__(self, idx:int):
        data = {}
        txt, _, emotion = pd.read_csv(self.txt_folder[idx]).iloc[0].values
    
        # Txt File
        txt, txt_mask = self.get_hub_txt(txt_path=txt, encoding=self.TEXT_ENCODING)
        data["txt"] = txt
        data["txt_mask"] = txt_mask
    
        # emotion 
        data["emotion"] = self.get_emo(emotion)

        sampling_rate, wav, wav_mask = self.get_wav(wav_path=self.wav_folder[idx])
        data["sampling_rate"] = sampling_rate
        data["wav"] = wav
        data["wav_mask"] = wav_mask

        return data
